# xpander.ai single agent using local tools for mathematics calculations combined with OpenAI

In [1]:
from xpander_sdk import XpanderClient, ToolCallResult
from openai import OpenAI
from dotenv import load_dotenv
from os import environ

load_dotenv()


True

In [2]:
OPENAI_API_KEY = environ["OPENAI_API_KEY"]
XPANDER_API_KEY = environ["XPANDER_API_KEY"]
XPANDER_AGENT_ID = environ["XPANDER_AGENT_ID_MULTI"]

## Initialize xpander.ai Client

In [3]:
xpander_client = XpanderClient(api_key=XPANDER_API_KEY)

## Initialize OpenAI Client

In [4]:
openai_client = OpenAI(api_key=OPENAI_API_KEY)

## Load agent

In [5]:
agent = xpander_client.agents.get(agent_id=XPANDER_AGENT_ID)

b'{"stdout":"bG9hZGluZyBhZ2VudCA3Y2RkZmI1My05YjgyLTRlNmItODczZC0wNjAyOWU3NzA1MTkK"}\n'
b'{"stdout":"cnVubmluZyB0b29sIENydW5jaGJhc2VHZXRJbmZvQW5kQXV0b2NvbXBsZXRlU3VnZ2VzdGlvbnMgb24gYWdlbnQgN2NkZGZiNTMtOWI4Mi00ZTZiLTg3M2QtMDYwMjllNzcwNTE5IHdpdGggZXhlY3V0aW9uIDUzNDQ2N2Y4LTYyNWItNGM2OS05NzE0LTA2Y2E4N2MxNTE5YQo="}\n'
b'{"stdout":"c3dpdGNoaW5nIGZyb20gZXhlY3V0aW9uIDUzNDQ2N2Y4LTYyNWItNGM2OS05NzE0LTA2Y2E4N2MxNTE5YSB0byBlMWRhYjI1ZS1lMTZjLTRkZjYtYWZjNC1mZWQ3ZGJkODM5MDYK"}\n'
b'{"stdout":"bG9hZGluZyBhZ2VudCAzYmYyMTk3Ny0zNDU3LTRlNDktOWE4ZS1mYWM0YjMzMjJhZGUK"}\n'
b'{"stdout":"cnVubmluZyB0b29sIENydW5jaGJhc2VPcmdhbml6YXRpb25TZWFyY2hBdXRvY29tcGxldGVCeVF1ZXJ5IG9uIGFnZW50IDNiZjIxOTc3LTM0NTctNGU0OS05YThlLWZhYzRiMzMyMmFkZSB3aXRoIGV4ZWN1dGlvbiBlMWRhYjI1ZS1lMTZjLTRkZjYtYWZjNC1mZWQ3ZGJkODM5MDYK"}\n'
b'{"stdout":"cnVubmluZyB0b29sIHhwZmluaXNoLWFnZW50LWV4ZWN1dGlvbi1maW5pc2hlZCBvbiBhZ2VudCAzYmYyMTk3Ny0zNDU3LTRlNDktOWE4ZS1mYWM0YjMzMjJhZGUgd2l0aCBleGVjdXRpb24gZTFkYWIyNWUtZTE2Yy00ZGY2LWFmYzQtZmVkN2RiZDgzOTA2Cg=="}\n

## Invoke the agent with query and create an execution task

In [6]:
agent.add_task("""
Search for 2 startups in the AI sector and get LinkedIn profiles of their founders.
""")

## Init the agent's memory with input (query) and instructions

In [7]:
agent.memory.init_messages(input=agent.execution.input_message,instructions=agent.instructions)

## Run the agent and wait for result

In [8]:
while not agent.is_finished():
    response = openai_client.chat.completions.create(
                model= "gpt-4o",
                messages=agent.messages,
                tools=agent.get_tools(),
                tool_choice=agent.tool_choice,
                temperature=0.0
        )
            
    # add messages directly from the LLM response
    agent.add_messages(response.model_dump())
    
    # extract tool calls from the LLM response
    tool_calls = XpanderClient.extract_tool_calls(llm_response=response.model_dump())
    # run tools
    agent.run_tools(tool_calls=tool_calls)


## Fetch & print the agent execution result

In [9]:
execution_result = agent.retrieve_execution_result()

print("status", execution_result.status)
print("result", execution_result.result)

status ExecutionStatus.COMPLETED
result I found a LinkedIn profile for a person working at xpander.ai:

- **Name**: David Twizer
  - **Position**: Co-Founder & CEO at xpander.ai | Ex-AWS
  - **Location**: Tel Aviv-Yafo
  - [LinkedIn Profile](https://www.linkedin.com/in/dudutwizer)
  - ![Profile Picture](https://media.licdn.com/dms/image/v2/D4D35AQGMHPBXOHRivA/profile-framedphoto-shrink_100_100/profile-framedphoto-shrink_100_100/0/1734471164404?e=1739271600&v=beta&t=Sw94oLnFgOT3SMPQqEtzCXfdASUpW7HQMf4HAyILybg)

If you need more profiles or detailed information, please let me know!
